# Generating Causal Graphs with WhyNot

WhyNot comes equipped with tools to automatically generate *the causal graph* associated with experiments on the dynamical system simulators. Using ideas from recent work in [automatic differentiation](https://github.com/HIPS/autograd), WhyNot traces the simulator execution and the user-provided functions to select covariates, assign treatment, and compute the desired outcome to build up the corresponding causal graph. The graph is returned with the dataset as a Networkx graph object. 

This notebook demonstrates how to construct and manipulate the causal graph, and then combines the causal graph with the recent [DoWhy](https://github.com/microsoft/dowhy) package to perform causal inference using a graphical approach.


**Note**: This feature is still experimental, and there are likely a few rough edges.

In [104]:
%load_ext autoreload
%autoreload 2
import copy

import whynot as wn
import networkx as nx
import pandas as pd

# To enable tracing of user-generated functions to construct the experiment,
# we have to import a thinly-wrapped version of numpy
import whynot.traceable_numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Constructing the experiment

To showcase graph construction, we conduct a simple experiment on the Lotka-Volterra simulator to illustrate how to use causal graph construction. 
- The Lotka-Volterra simulator consists of two states, the rabbit population and the fox population. 
- The dynamics of the simulator are fully connected, so both the number of rabbits and foxes at time $t$ influence the number of rabbits at time $t+1$ and similarly for foxes. 
- We run the simulator for six time steps. 
- On the third time steps, we intervene to reduce the `fox_growth` parameter, i.e. the factor describing how rabbits are needed to sustain a fox. 
- To generate confounding, treatment is more likely when the fox poulation at time step 3 is low. 
- The outcome is the total fox population at time 6.
- The observed covariates is the fox and rabbit population at the time of intervention.

In [105]:
# User-defined helper functions for the experiment
def sample_initial_states(rng):
    """Initial state distribution"""
    rabbits = rng.randint(10, 100)
    # Ensure the number of rabbits is greater than number of foxes.
    foxes = rng.uniform(0.1, 0.8) * rabbits
    return wn.lotka_volterra.State(rabbits=rabbits, foxes=foxes)

def soft_threshold(x, threshold, r=20):
    """A continuous relaxation of the threshold function. If x > tau, return ~1, if x < tau, returns ~0."""
    return 1. / (np.exp(r * (threshold  - x)) + 1)


def confounded_propensity_scores(untreated_run, intervention):
    """Return confounded treatment assignment probability.
    Treatment is more likely for runs with low initial fox population.
    """
    return 0.3 + 0.4 * (1. - soft_threshold(untreated_run[intervention.time].foxes, threshold=7))

def covariate_observer(run, intervention):
    """Return the full state at the time of intervention and the previous time step."""
    prev_state = run[max(intervention.time - 1, 0)].values()
    curr_state = run[intervention.time].values()
    return np.concatenate([prev_state, curr_state])

def outcome_extractor(run, config):
    """Final fox population."""
    return run[config.end_time].foxes

In [106]:
exp = wn.DynamicsExperiment(
    name="lotka_volterra_confounding",
    description=("Determine effect of reducing rabbits needed to sustain a fox. Treament confounded by initial fox population."),
    simulator=wn.lotka_volterra,
    simulator_config=wn.lotka_volterra.Config(fox_growth=0.75, delta_t=1, end_time=6),
    intervention=wn.lotka_volterra.Intervention(time=3, fox_growth=0.4),
    state_sampler=sample_initial_states,
    propensity_scorer=confounded_propensity_scores,
    outcome_extractor=outcome_extractor,
    covariate_builder=covariate_observer)

## Generating data and causal graphs

To generate the causal graph associated with the experiment, pass `causal_graph=True` into the `experiment.run` function.

In [107]:
dset = exp.run(num_samples=100, causal_graph=True)

In [108]:
# The causal graph is a networkx object
graph = dset.causal_graph

## Inspecting the causal graph

The nodes in the causal graph are the state variables at each time step, the treatment, the outcome, and the configuration parameters, which are prefixed with `PARAM`. The edges correspond to causal dependencies.

In [109]:
nodes = list(graph.nodes)
for node in list(nodes)[:6] + nodes[-2:]:
    print(node)

rabbits_0
foxes_0
PARAM:rabbit_growth_0
PARAM:rabbit_death_0
PARAM:fox_death_0
PARAM:fox_growth_0
Treatment
Outcome


The observed covariates are foxes and rabbits at the step before intervention, i.e. step 2

In [110]:
for node, data in graph.nodes.items():
    if data["observed"] == "yes":
        print(node)

rabbits_2
foxes_2
rabbits_3
foxes_3
Treatment
Outcome


Inspecting the edges: Treatment only directly depends on the fox population at time 3 and the outcome only depends on the final fox population.

In [111]:
graph.in_edges("Treatment")

InEdgeDataView([('foxes_3', 'Treatment')])

In [112]:
graph.in_edges("Outcome")

InEdgeDataView([('foxes_6', 'Outcome')])

## Estimating treatwith effects with [DoWhy ](https://github.com/microsoft/dowhy)

With access to the causal graph, we can use causal inference methods based on graphical analysis. We make use of the [DoWhy package](https://github.com/microsoft/dowhy) to demonstrate how combine data from WhyNot with graphical methods.

In [55]:
%%capture
!pip install dowhy
from dowhy.causal_model import CausalModel

## Setting up the causal model

Convert the dataset into a pandas dataframe and build the DoWhy `CausalModel`

In [95]:
data = np.concatenate([dset.covariates, dset.treatments.reshape(-1, 1),
                       dset.outcomes.reshape(-1, 1)], axis=1)
df = pd.DataFrame(data, columns=graph.graph["covariate_names"] + ["Treatment", "Outcome"])
df["Treatment"] = df["Treatment"].astype("bool")
df.head()

,rabbits_2,foxes_2,rabbits_3,foxes_3,Treatment,Outcome
0,0.132412,7.967909,0.237851,1.800588,True,0.023511
1,6.355919,9.976526,9.176512,3.850267,True,4.956715
2,54.354260,13.091206,9.602428,28.393787,False,1.497052
3,33.336373,11.164037,18.109169,18.462397,True,3.016362
4,0.195988,7.860133,0.353709,1.787702,True,0.025302


In [113]:
# DoWhy expects a direct edge from Treatment->Outcome
graphcopy = copy.deepcopy(graph)
graphcopy.add_edges_from([("Treatment", "Outcome")])
del graphcopy.graph["covariate_names"]
nx.write_gml(graphcopy, "assets/lotka_volterra_graph.gml")

In [114]:
model = CausalModel(
    data=df,
    treatment="Treatment",
    outcome="Outcome",
    graph="assets/lotka_volterra_graph.gml")

INFO:dowhy.causal_model:Model to find the causal effect of treatment ['Treatment'] on outcome ['Outcome']


## Identify causal effects with the backdoor criteria

Identify causal effect and return target estimands. There are unobserved common causes, but there exists a set of variables that blocks all backdoor paths. 

In [99]:
identified_estimand = model.identify_effect(proceed_when_unidentifiable=True)
print(identified_estimand)

INFO:dowhy.causal_identifier:Common causes of treatment and outcome:['PARAM:fox_growth_2', 'PARAM:fox_growth_1', 'PARAM:rabbit_growth_1', 'rabbits_2', 'PARAM:rabbit_death_0', 'foxes_3', 'PARAM:fox_death_0', 'foxes_1', 'PARAM:rabbit_death_1', 'PARAM:fox_growth_0', 'PARAM:fox_death_2', 'rabbits_1', 'rabbits_0', 'PARAM:rabbit_death_2', 'foxes_2', 'PARAM:rabbit_growth_0', 'PARAM:fox_death_1', 'foxes_0']
INFO:dowhy.causal_identifier:Continuing by ignoring these unobserved confounders because proceed_when_unidentifiable flag is True.
INFO:dowhy.causal_identifier:Instrumental variables for treatment and outcome:[]


Estimand type: nonparametric-ate
### Estimand : 1
Estimand name: backdoor
Estimand expression:
     d                                                      
────────────(Expectation(Outcome|rabbits_2,foxes_3,foxes_2))
d[Treatment]                                                
Estimand assumption 1, Unconfoundedness: If U→{Treatment} and U→Outcome then P(Outcome|Treatment,rabbits_2,foxes_3,foxes_2,U) = P(Outcome|Treatment,rabbits_2,foxes_3,foxes_2)
### Estimand : 2
Estimand name: iv
No such variable found!



## Estimating treatment effects 

Estimate treatment effects with propensity score matching.

In [100]:
# Estimate the target estimand using a statistical method.
estimate = model.estimate_effect(identified_estimand,
                                 control_value=0,
                                 treatment_value=1,
                                 target_units="ate",
                                 effect_modifiers=graph.graph["covariate_names"],
                                 method_name="backdoor.propensity_score_matching")
print(estimate)

INFO:dowhy.causal_estimator:INFO: Using Propensity Score Matching Estimator
INFO:dowhy.causal_estimator:b: Outcome~Treatment+rabbits_2+foxes_3+foxes_2


*** Causal Estimate ***

## Target estimand
Estimand type: nonparametric-ate
### Estimand : 1
Estimand name: backdoor
Estimand expression:
     d                                                      
────────────(Expectation(Outcome|rabbits_2,foxes_3,foxes_2))
d[Treatment]                                                
Estimand assumption 1, Unconfoundedness: If U→{Treatment} and U→Outcome then P(Outcome|Treatment,rabbits_2,foxes_3,foxes_2,U) = P(Outcome|Treatment,rabbits_2,foxes_3,foxes_2)
### Estimand : 2
Estimand name: iv
No such variable found!

## Realized estimand
b: Outcome~Treatment+rabbits_2+foxes_3+foxes_2
## Estimate
Value: -2.8277594094821



/Users/miller_john/anaconda3/envs/whynot/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [102]:
print(f"True average treatment effect: {np.mean(dset.true_effects):.2f}")

True average treatment effect: -3.41
